In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

raw = pd.read_csv('E:\\private\\学习\\上课资料\\数据\\商品购买预测数据集\\train.csv')

In [2]:
raw

,order_detail_id,order_id,order_total_num,order_amount,order_total_payment,order_total_discount,order_pay_time,order_status,order_count,is_customer_rate,...,customer_gender,member_status,is_member_actived,goods_id,goods_class_id,goods_price,goods_status,goods_has_discount,goods_list_time,goods_delist_time
0,1000000,1000000,1.0,239.9,96.9,0.0,2012-11-01 00:10:56,6,1.0,0.0,...,NaN,NaN,NaN,998,998,54.909289,1.0,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07
1,1001530,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1953,1953,45.961352,0.0,1.0,2013-08-28 17:27:50,2013-09-01 00:38:17
2,1001531,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1083,1083,53.035439,1.0,0.0,2014-10-29 18:21:05,2014-11-05 18:21:05
3,1001532,1001328,3.0,180.0,89.7,0.0,2013-08-31 22:06:35,6,1.0,0.0,...,NaN,NaN,NaN,1013,1013,46.046917,1.0,1.0,2014-10-25 11:00:00,2014-11-01 11:00:00
4,1001533,1001329,1.0,159.9,65.9,0.0,2013-08-31 21:33:36,6,1.0,0.0,...,NaN,NaN,NaN,1628,1628,50.722161,1.0,0.0,2014-10-23 15:35:33,2014-10-30 15:35:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2306866,3685495,3238358,1.0,199.0,59.9,0.0,2013-01-10 19:24:31,6,1.0,0.0,...,0.0,1.0,1.0,1173,1173,53.012016,1.0,0.0,2014-10-25 10:14:59,2014-11-01 10:14:59
2306867,3685496,3238359,2.0,299.8,89.9,0.0,2013-01-27 15:00:27,6,2.0,0.0,...,0.0,1.0,1.0,2513,2513,42.693822,2.0,0.0,2014-01-10 15:56:40,2014-01-11 12:46:19
2306868,3685497,3238359,2.0,299.8,89.9,0.0,2013-01-27 15:00:27,6,2.0,0.0,...,0.0,1.0,1.0,998,998,54.889036,1.0,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07
2306869,3685498,3238360,1.0,168.0,76.9,0.0,2012-11-11 00:10:37,6,1.0,0.0,...,0.0,1.0,1.0,1423,1423,52.078004,1.0,0.0,2014-10-30 09:31:53,2014-11-06 09:31:53


In [3]:
raw.columns
raw.isnull().sum()
#缺失值信息如下：
#customer_province缺失            1139
#customer_city缺失                1150
#customer_gender缺失           1671081
#member_status缺失             1671081
#is_member_actived缺失         1671081
#goods_price缺失                   436
raw['customer_gender'].value_counts()

0.0    603970
2.0     18468
1.0     13352
Name: customer_gender, dtype: int64

In [4]:
print(raw['goods_id'].value_counts())
print(len(raw['goods_id'].unique()))
#对Goods_id进行标签编码
raw['goods_id'] = pd.factorize(raw['goods_id'])[0]
raw['goods_id']

998     472943
1038    171277
1333     91748
1413     67783
1488     60263
         ...  
1918         1
6263         1
6248         1
6033         1
6143         1
Name: goods_id, Length: 1037, dtype: int64
1037


0            0
1            1
2            2
3            3
4            4
          ... 
2306866    115
2306867    230
2306868      0
2306869     43
2306870     10
Name: goods_id, Length: 2306871, dtype: int64

In [5]:
raw['customer_province'].value_counts()
raw['customer_city'].value_counts()

上海市      117625
北京市       71394
广州市       59772
杭州市       53752
武汉市       47894
          ...  
高雄市           1
可克达拉市         1
台中市           1
宜兰县           1
新北市           1
Name: customer_city, Length: 375, dtype: int64

In [6]:
#训练集：截止到7月31号，验证集8月
raw['order_pay_time'].max()

'2013-08-31 23:59:59'

In [23]:
%%time
import datetime
#对数据进行预处理
def preprocess(raw, train='train'):
    #缺失值信息如下：
    #customer_province缺失            1139
    #customer_city缺失                1150
    #customer_gender缺失           1671081
    #member_status缺失             1671081
    #is_member_actived缺失         1671081
    #goods_price缺失                   436
    #对原始数据按customer_id聚合，customer_gender进行填充
    data = pd.DataFrame(raw.groupby('customer_id')['customer_gender'].last().fillna(0))
    #添加新的列， 针对每个用户，最后一次的行为
    #商品相关（最后一次行为）
    data[['goods_id_last','goods_status_last','goods_price_last','goods_has_discount_last','goods_list_time_last',\
         'goods_delist_time_last']] = raw.groupby(['customer_id'])['goods_id','goods_status','goods_price','goods_has_discount', \
                                                                   'goods_list_time','goods_delist_time'].last()
    #按照订单相关（最后一次行为）
    data[['order_total_num_last','order_amount_last','order_total_payment_last','order_total_discount_last', \
         'order_pay_time_last','order_status_last','order_count_last','is_customer_rate_last','order_detail_status_last', \
         'order_detail_goods_num_last','order_detail_amount_last','order_detail_payment_last', \
          'order_detail_discount_last']] = raw.groupby('customer_id')['order_total_num','order_amount','order_total_payment','order_total_discount','order_pay_time', \
                     'order_status','order_count','is_customer_rate','order_detail_status', 'order_detail_goods_num', \
                     'order_detail_amount','order_detail_payment','order_detail_discount'].last()
    
    #添加商品原始价格（多种统计字段）std,mean,min,max
    data[['goods_price_std','goods_price_mean','goods_price_min','goods_price_max']] = raw.groupby('customer_id').agg({'goods_price':['std','mean','min','max']})
    #订单的实际金额（多种统计字段）std，mean，min，max
    data[['order_detail_payment_std','order_detail_payment_mean','order_detail_payment_min','order_detail_payment_max']] = raw.groupby('customer_id').agg({'order_detail_payment':['std','mean','min','max']})
    #用户购买数量
    data['count'] = raw.groupby('customer_id')['order_id'].nunique()
    #用户购买的商品数量
    data['goods_count'] = raw.groupby('customer_id')['goods_id'].nunique()
    #用户所在的城市和省份
    data['customer_province'] = raw.groupby('customer_id')['customer_province'].last()
    data['customer_city'] = raw.groupby('customer_id')['customer_city'].last()
    #订单的付款时间
    def time2multi(x):
        t = datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
        return pd.Series([t.month, t.day, t.weekday(), t.hour,t.minute, t.second])
    data[['order_pay_time_m','order_pay_time_d','order_pay_time_week','order_pay_time_h','order_pay_time_min','order_pay_time_s']] = data['order_pay_time_last'].apply(time2multi)
    t_str = '2013-01-01 00:00:00'
    t = datetime.datetime.strptime(t_str, '%Y-%m-%d %H:%M:%S')
    data['good_list_time_diff'] = data['goods_list_time_last'].map(lambda x :(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') -t).days/365)
    data['good_delist_time_diff'] = data['goods_delist_time_last'].map(lambda x :(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') -t).days/365)
    #商品展示时间
    data['goods_diff'] = data['good_delist_time_diff'] - data['good_list_time_diff']
    #用户是否评价统计，mean，sum
    data[['is_customer_rate_mean','is_customer_rate_sum']] = raw.groupby('customer_id').agg({'is_customer_rate':['mean','sum']})
    #用户购买的goods数量
    data['order_total_num_sum'] = raw.groupby('customer_id')['order_total_num'].sum()
    #商品折扣统计属性（sum, mean）
    data[['goods_has_discount_mean','goods_has_discount_sum']] =  raw.groupby('customer_id').agg({'goods_has_discount':['mean','sum']})
    return data

#8月之前的数据作为训练集
train_raw = raw[raw['order_pay_time'] < '2013-07-31 23:59:59']
train_raw = preprocess(train_raw.iloc[1671220:1681220, :])
#train_raw = preprocess(train_raw)
train_raw

Wall time: 2.04 s


,customer_gender,goods_id_last,goods_status_last,goods_price_last,goods_has_discount_last,goods_list_time_last,goods_delist_time_last,order_total_num_last,order_amount_last,order_total_payment_last,...,order_pay_time_min,order_pay_time_s,good_list_time_diff,good_delist_time_diff,goods_diff,is_customer_rate_mean,is_customer_rate_sum,order_total_num_sum,goods_has_discount_mean,goods_has_discount_sum
customer_id,,,,,,,,,,,,,,,,,,,,,
1151118,0.0,0,1.0,54.813352,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07,1.0,239.9,89.90,...,0,30,1.813699,1.832877,0.019178,0.0,0.0,1.0,0.0,0.0
1480123,0.0,0,1.0,54.839917,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07,1.0,239.9,89.90,...,0,29,1.813699,1.832877,0.019178,0.0,0.0,1.0,0.0,0.0
1565402,0.0,157,1.0,59.945166,0.0,2014-10-30 00:00:00,2014-11-06 00:00:00,1.0,399.9,155.90,...,30,44,1.827397,1.846575,0.019178,0.0,0.0,1.0,0.0,0.0
1649573,0.0,289,1.0,55.282611,0.0,2014-10-29 11:09:07,2014-11-05 11:09:07,4.0,877.9,166.60,...,13,25,1.824658,1.843836,0.019178,0.0,0.0,17.0,0.0,0.0
1781299,0.0,98,1.0,59.947711,0.0,2014-10-25 00:00:00,2014-11-01 00:00:00,1.0,399.9,133.10,...,7,10,1.813699,1.832877,0.019178,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2547992,0.0,73,1.0,54.372260,0.0,2014-10-28 18:47:06,2014-11-04 18:47:06,2.0,449.9,156.72,...,35,34,1.821918,1.841096,0.019178,0.0,0.0,4.0,0.0,0.0
2547993,0.0,82,1.0,48.814590,1.0,2014-10-29 14:20:39,2014-11-05 14:20:39,2.0,260.0,96.23,...,52,55,1.824658,1.843836,0.019178,0.0,0.0,2.0,1.0,1.0
2547994,0.0,14,1.0,53.087974,0.0,2014-10-24 09:19:42,2014-10-31 09:19:42,1.0,199.9,86.90,...,14,51,1.810959,1.830137,0.019178,0.0,0.0,1.0,0.0,0.0
